In [1]:
from pyspark.sql import SparkSession

In [2]:
%%sql
CREATE TABLE IF NOT EXISTS nyc.taxis;

24/11/14 13:38:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [3]:
%%sql

DROP TABLE IF EXISTS nyc.taxis;

++
||
++
++

In [4]:
%%sql

CREATE TABLE nyc.taxis (
    VendorID              bigint,
    tpep_pickup_datetime  timestamp,
    tpep_dropoff_datetime timestamp,
    passenger_count       double,
    trip_distance         double,
    RatecodeID            double,
    store_and_fwd_flag    string,
    PULocationID          bigint,
    DOLocationID          bigint,
    payment_type          bigint,
    fare_amount           double,
    extra                 double,
    mta_tax               double,
    tip_amount            double,
    tolls_amount          double,
    improvement_surcharge double,
    total_amount          double,
    congestion_surcharge  double,
    airport_fee           double
)
USING iceberg
PARTITIONED BY (days(tpep_pickup_datetime))

++
||
++
++

In [5]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2022-01.parquet")
df.writeTo("nyc.taxis").append()

In [6]:
%%sql
SELECT * FROM nyc.taxis.manifests

content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,s3://warehouse/nyc/taxis/metadata/d7166d89-f075-4c65-a15d-05f627ede544-m0.avro,16260,0,6022052716745643488,40,0,0,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2008-12-31', upper_bound='2022-05-18')]"


In [7]:
%%sql
SELECT * FROM nyc.taxis.files

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://warehouse/nyc/taxis/data/tpep_pickup_datetime_day=2022-01-09/00000-9-d0a54931-224d-4ebf-a262-f88f62864cad-0-00005.parquet,PARQUET,0,"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 9))",64014,1091013,"{1: 8149, 2: 290180, 3: 292847, 4: 13892, 5: 94087, 6: 5624, 7: 2726, 8: 45656, 9: 51113, 10: 9151, 11: 66115, 12: 14761, 13: 2402, 14: 71200, 15: 8085, 16: 1363, 17: 93609, 18: 6818, 19: 6269}","{1: 64014, 2: 64014, 3: 64014, 4: 64014, 5: 64014, 6: 64014, 7: 64014, 8: 64014, 9: 64014, 10: 64014, 11: 64014, 12: 64014, 13: 64014, 14: 64014, 15: 64014, 16: 64014, 17: 64014, 18: 64014, 19: 64014}","{1: 0, 2: 0, 3: 0, 4: 1352, 5: 0, 6: 1352, 7: 1352, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1352, 19: 1352}","{16: 0, 17: 0, 18: 0, 19: 0, 4: 0, 5: 0, 6: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}","{1: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'\x00`\xb4\xe6\x1a\xd5\x05\x00'), 3: bytearray(b'\x00\\a\xed\x1a\xd5\x05\x00'), 4: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 5: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 6: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 7: bytearray(b'N'), 8: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 9: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 10: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 11: bytearray(b'\x00\x00\x00\x00\x00\xf0t\xc0'), 12: bytearray(b'\x00\x00\x00\x00\x00\x00\x04\xc0'), 13: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0\xbf'), 14: bytearray(b'\x1f\x85\xebQ\xb8\x1e\xe5\xbf'), 15: bytearray(b""\x00\x00\x00\x00\x00\x80\'\xc0""), 16: bytearray(b'333333\xd3\xbf'), 17: bytearray(b'\xd7\xa3p=\noo\xc0'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04\xc0'), 19: bytearray(b'\x00\x00\x00\x00\x00\x00\xf4\xbf')}","{1: bytearray(b'\x06\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'\xc0}|\x04/\xd5\x05\x00'), 3: bytearray(b'@\\>\xe0K\xd5\x05\x00'), 4: bytearray(b'\x00\x00\x00\x00\x00\x00\x18@'), 5: bytearray(b'\x9a\x99\x99\x99\xd1$\x01A'), 6: bytearray(b'\x00\x00\x00\x00\x00\xc0X@'), 7: bytearray(b'Y'), 8: bytearray(b'\t\x01\x00\x00\x00\x00\x00\x00'), 9: bytearray(b'\t\x01\x00\x00\x00\x00\x00\x00'), 10: bytearray(b'\x04\x00\x00\x00\x00\x00\x00\x00'), 11: bytearray(b'\x00\x00\x00\x00\x00\xf0t@'), 12: bytearray(b'\x00\x00\x00\x00\x00\x00\x1c@'), 13: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 14: bytearray(b'\\\x8f\xc2\xf5(\x1ce@'), 15: bytearray(b'\x00\x00\x00\x00\x00@P@'), 16: bytearray(b'333333\xd3?'), 17: bytearray(b'\xcd\xcc\xcc\xcc\xcc\xf4t@'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04@'), 19: bytearray(b'\x00\x00\x00\x00\x00\x00\xf4?')}",None,[4],None,0,"Row(DOLocationID=Row(column_size=51113, value_count=64014, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=265), PULocationID=Row(column_size=45656, value_count=64014, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=265), RatecodeID=Row(column_size=5624, value_count=64014, null_value_count=1352, nan_value_count=0, lower_bound=1.0, upper_bound=99.0), VendorID=Row(column_size=8149, value_count=64014, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=6), airport_fee=Row(column_size=6269, value_count=64014, null_value_count=1352, nan_value_count=0, lower_bound=-1.25, upper_bound=1.25), congestion_surcharge=Row(column_size=6818, value_count=64014, null_value_count=1352, nan_value_count=0, lower_bound=-2.5, upper_bound=2.5), extra=Row(column_size=14761, value_count=64014, null_value_count=0, nan_value_count=0, lower_bound=-2.5, upper_bound=7.0), fare_amount=Row(column_size=66115, value_count=64014, null_value_count=0, nan_value_count=0, lower_bound=-335.0, upper_bound=335.0), improvement_surcharge=Row(column_size=1363, value_count=64014, null_value_count=0, nan_value_count=0, lower_bound=

In [8]:
%%sql
select * from nyc.taxis.partitions

partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
"Row(tpep_pickup_datetime_day=datetime.date(2021, 12, 31))",0,24,1,7075,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 1))",0,63441,1,1095030,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 3, 9))",0,1,1,5528,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 4))",0,74562,1,1236592,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 5))",0,74592,1,1232453,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 2))",0,58421,1,1014882,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 3))",0,72405,1,1214233,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 8))",0,83177,1,1355096,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 9))",0,64014,1,1091013,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 6))",0,79909,1,1299518,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488


In [9]:
%%sql

ALTER TABLE nyc.taxis DROP PARTITION FIELD days(tpep_pickup_datetime)

++
||
++
++

In [10]:
%%sql

ALTER TABLE nyc.taxis ADD PARTITION FIELD hours(tpep_pickup_datetime)

++
||
++
++

In [11]:
%%sql

SELECT * FROM nyc.taxis.partitions

partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 22))",0,96587,1,1529503,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 8))",0,83177,1,1355096,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2009, 1, 1))",0,8,1,6599,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 19))",0,86640,1,1375640,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 30))",0,71229,1,1194244,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 5))",0,74592,1,1232453,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 16))",0,72783,1,1214450,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 2))",0,58421,1,1014882,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 27))",0,99766,1,1564617,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488
"Row(tpep_pickup_datetime_day=datetime.date(2022, 1, 6))",0,79909,1,1299518,0,0,0,0,2024-11-14 13:38:35.176000,6022052716745643488


In [12]:
%%sql

CALL system.rewrite_data_files('nyc.taxis')

rewritten_data_files_count,added_data_files_count,rewritten_bytes_count,failed_data_files_count
40,758,40057638,0


In [13]:
%%sql

SELECT *
FROM nyc.taxis.files


content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://warehouse/nyc/taxis/data/tpep_pickup_datetime_hour=2022-04-06-04/00199-224-a95b0db0-f62b-47ad-a99e-a8423f0fb077-0-00007.parquet,PARQUET,2,"Row(tpep_pickup_datetime_day=None, tpep_pickup_datetime_hour=458116)",1,5528,"{1: 46, 2: 46, 3: 46, 4: 46, 5: 46, 6: 46, 7: 43, 8: 46, 9: 46, 10: 46, 11: 46, 12: 46, 13: 46, 14: 46, 15: 46, 16: 46, 17: 46, 18: 46, 19: 46}","{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1}","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0}","{16: 0, 17: 0, 18: 0, 19: 0, 4: 0, 5: 0, 6: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}","{1: bytearray(b'\x02\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'@\xc7\xe1\x86\xf4\xdb\x05\x00'), 3: bytearray(b'\xc0%\xe7\xa7\xf4\xdb\x05\x00'), 4: bytearray(b'\x00\x00\x00\x00\x00\x00\x00@'), 5: bytearray(b'\xb8\x1e\x85\xebQ\xb8\xfa?'), 6: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 7: bytearray(b'N'), 8: bytearray(b'\xa1\x00\x00\x00\x00\x00\x00\x00'), 9: bytearray(b'\xea\x00\x00\x00\x00\x00\x00\x00'), 10: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 11: bytearray(b'\x00\x00\x00\x00\x00\x00!@'), 12: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 13: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 14: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 15: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 16: bytearray(b'333333\xd3?'), 17: bytearray(b'\x9a\x99\x99\x99\x99\x99)@'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04@'), 19: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00')}","{1: bytearray(b'\x02\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'@\xc7\xe1\x86\xf4\xdb\x05\x00'), 3: bytearray(b'\xc0%\xe7\xa7\xf4\xdb\x05\x00'), 4: bytearray(b'\x00\x00\x00\x00\x00\x00\x00@'), 5: bytearray(b'\xb8\x1e\x85\xebQ\xb8\xfa?'), 6: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 7: bytearray(b'N'), 8: bytearray(b'\xa1\x00\x00\x00\x00\x00\x00\x00'), 9: bytearray(b'\xea\x00\x00\x00\x00\x00\x00\x00'), 10: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 11: bytearray(b'\x00\x00\x00\x00\x00\x00!@'), 12: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 13: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 14: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 15: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 16: bytearray(b'333333\xd3?'), 17: bytearray(b'\x9a\x99\x99\x99\x99\x99)@'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04@'), 19: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00')}",None,[4],None,0,"Row(DOLocationID=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=234, upper_bound=234), PULocationID=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=161, upper_bound=161), RatecodeID=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=1.0, upper_bound=1.0), VendorID=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=2, upper_bound=2), airport_fee=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=0.0, upper_bound=0.0), congestion_surcharge=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=2.5, upper_bound=2.5), extra=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=0.0, upper_bound=0.0), fare_amount=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=8.5, upper_bound=8.5), improvement_surcharge=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=0.3, upper_bound=0.3), mta_tax=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=0.5, upper_bound=0.5), passenger_count=Row(column_siz

In [14]:
%%sql

SELECT *
FROM nyc.taxis.snapshots
ORDER BY committed_at DESC

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2024-11-14 13:38:56.101000,5377373251962650806,6022052716745643488,replace,s3://warehouse/nyc/taxis/metadata/snap-5377373251962650806-1-bf050da6-d035-44c7-88ba-c0487eb7dde5.avro,"{'engine-version': '3.5.3', 'added-data-files': '758', 'total-equality-deletes': '0', 'app-id': 'local-1731591497860', 'added-records': '2463931', 'deleted-data-files': '40', 'deleted-records': '2463931', 'total-records': '2463931', 'removed-files-size': '40057638', 'changed-partition-count': '798', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '46539857', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.6.0 (commit 229d8f6fcd109e6c8943ea7cbb41dab746c6d0ed)', 'total-files-size': '46539857', 'total-data-files': '758'}"
2024-11-14 13:38:35.176000,6022052716745643488,None,append,s3://warehouse/nyc/taxis/metadata/snap-6022052716745643488-1-d7166d89-f075-4c65-a15d-05f627ede544.avro,"{'engine-version': '3.5.3', 'added-data-files': '40', 'total-equality-deletes': '0', 'app-id': 'local-1731591497860', 'added-records': '2463931', 'total-records': '2463931', 'spark.app.id': 'local-1731591497860', 'changed-partition-count': '40', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '40057638', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.6.0 (commit 229d8f6fcd109e6c8943ea7cbb41dab746c6d0ed)', 'total-files-size': '40057638', 'total-data-files': '40'}"
